In [1]:
import re
import altair as alt
import numpy as np
import pandas as pd
import scipy.stats
import httpimport

# allow more rows for Altair
_ = alt.data_transformers.disable_max_rows()


In [2]:
# Import custom altair theme from remote github using httpimport module
def import_theme_new():
    with httpimport.github_repo("bblarsen-sci", "altair_themes", "main"):
        import main_theme

        @alt.theme.register("custom_theme", enable=True)
        def custom_theme():
            return main_theme.main_theme()


import_theme_new()


In [3]:
escape_df = pd.read_csv(
    "../../results/filtered_data/antibody_escape/combined/all_antibodies_escape_filtered.csv"
)
ic50 = pd.read_csv("../../data/experimental_data/relative_validation_IC50s.csv")

ic50['wildtype'] = ic50['Mutant'].str[0]
ic50["site"] = ic50["Mutant"].str.extract(r"([0-9]+)").astype(int)
ic50['mutant'] = ic50['Mutant'].str[-1]

merged_df = pd.merge(
    ic50,
    escape_df,
    how='left',
    on=['antibody','site', 'wildtype', 'mutant', 'site'],
)
display(merged_df)

KeyError: 'Mutant'

In [ ]:
merged_df["relative_ic50"] = (merged_df["IC50_mutant"] / merged_df["IC50_unmutated"]).round(1)

# Calculate log2 relative ic50 values for each mutation relative to mean unmutated value
merged_df["log2_relative_ic50"] = np.log2(merged_df["relative_ic50"]).round(1)

In [ ]:
antibody_order = [
    "12B2",
    "2D3",
    "4H3",
    "1A9",
    "1F2",
    "2B12",
]

customColors = [
    "#5778a4",
    "#e49444",
    "#d1615d",
    "#85b6b2",
    "#6a9f58",
    "#e7ca60",
    "#a87c9f",
    "#f1a2a9",
    "#967662",
    "#b8b0ac",
]

In [ ]:
# now lets load in the icXX estimates
#antibodies = merged_df['antibody'].unique().tolist()
antibodies = ["12B2", "2D3", "4H3", "1A9", "1F2", "2B12"]
#print(antibodies)

empty = []
for antibody in antibodies:
    tmp_df = pd.read_csv(f"../../../results/antibody_escape/averages/{antibody}_mut_icXX.csv").assign(antibody=antibody)
    empty.append(tmp_df)

merged_icXX = pd.concat(empty)
#display(merged_icXX)


In [ ]:
merged_ic_df = pd.merge(
    ic50,
    merged_icXX,
    how="left",
    on=["antibody", "site", "wildtype", "mutant"],
)

merged_ic_df["relative_ic50"] = (
    merged_ic_df["IC50_mutant"] / merged_ic_df["IC50_unmutated"]
).round(1)

# Calculate log2 relative ic50 values for each mutation relative to mean unmutated value
merged_ic_df["log2_relative_ic50"] = np.log2(merged_ic_df["relative_ic50"]).round(1)

In [ ]:
merged_ic_df = pd.read_csv('../../data/experimental_data/icXX_merged.csv')
#display(merged_ic_df)

chart = (
    alt.Chart(merged_df)
    .mark_point(size=120, opacity=1, filled=True, stroke="black", strokeWidth=1)
    .encode(
        x=alt.X(
            "escape_mean",
            title="Escape by DMS",
            scale=alt.Scale(domain=[0, 10]),
        ),
        y=alt.Y("log2_relative_ic50", title="Relative IC50"),
        color=alt.Color(
            "antibody",
            scale=alt.Scale(domain=antibody_order, range=customColors),
        ),
        shape=alt.Shape(
            "threshold",
            # scale=alt.Scale(domain=["True", "False"], range=["triangle", "circle"]),
        ),
    )
    .properties(width=200, height=200)
)
#chart.display()

ic_chart = (
    alt.Chart(merged_ic_df)
    .mark_point(size=120, opacity=1, filled=True, stroke='black', strokeWidth=1)
    .encode(
        x=alt.X("dms_log2_ic50", title="Change in IC50 by DMS", scale=alt.Scale(domain=[0, 10])),
        y=alt.Y("log2_relative_ic50", title="Relative IC50"),
        color=alt.Color(
            "antibody",
            scale=alt.Scale(domain=antibody_order, range=customColors),
        ),
        shape=alt.Shape(
            "threshold",
            #scale=alt.Scale(domain=["True", "False"], range=["triangle", "circle"]),
        ),
    )
    .properties(width=200, height=200)
)
#ic_chart.display()

combined_chart = alt.hconcat(chart, ic_chart).resolve_scale(color='shared', x='shared', y='shared')
combined_chart.display()
#combined_chart.save("../../icXX_validation.png", ppi=300)
#combined_chart.save("../../icXX_validation.svg")
